|||
|---|---|
|Title|nwtimetracking|
|Author|numbworks|
|Version|5.0.2|
||Please check [docs/docs-nwtimetracking.md](../docs/docs-nwtimetracking.md) before proceeding.|

#### Global Modules

#### Local Modules

In [1]:
from nwtimetracking import SettingBag, ComponentBag, TimeTrackingProcessor, OPTION

#### Functions : Temp

#### Settings

In [2]:
setting_bag : SettingBag = SettingBag(
    options_tt = [OPTION.display],
    options_tt_latest_five = [OPTION.display],
    options_tts_by_month = [OPTION.display],
    options_tts_by_year = [OPTION.display],
    options_tts_by_range = [OPTION.display],
    options_tts_by_spn = [OPTION.display],
    options_tts_by_spv = [OPTION.display],
    options_tts_by_hashtag_year = [OPTION.display],
    options_tts_by_hashtag = [OPTION.display],
    options_tts_by_year_month_spnv = [OPTION.display],
    options_tts_by_timeranges = [OPTION.display],
    options_ttd_effort_status = [OPTION.display],
    excel_nrows = 1804
)

#### Main : Analysis

In [3]:
tt_processor : TimeTrackingProcessor = TimeTrackingProcessor(component_bag = ComponentBag(), setting_bag = setting_bag)
tt_processor.initialize()
tt_processor.process_tt_latest_five()
tt_processor.process_tts_by_month()
tt_processor.process_tts_by_year()
tt_processor.process_tts_by_range()
tt_processor.process_tts_by_spn()
tt_processor.process_tts_by_spv()
tt_processor.process_tts_by_hashtag_year()
tt_processor.process_tts_by_hashtag()
tt_processor.process_tts_by_year_month_spnv()
tt_processor.process_tts_by_timeranges()

Date,StartTime,EndTime,Effort,Hashtag,Descriptor,IsSoftwareProject,IsReleaseDay,Year,Month
2025-12-21,18:45,20:45,2h 00m,#python,nwreadinglist v4.4.0,True,False,2025,12
2025-12-21,21:45,22:45,1h 00m,#python,nwreadinglist v4.4.0,True,False,2025,12
2025-12-21,23:45,00:00,0h 15m,#python,nwreadinglist v4.4.0,True,False,2025,12
2025-12-22,13:15,22:00,8h 45m,#python,nwreadinglist v4.4.0,True,True,2025,12
2025-12-22,23:00,00:30,1h 30m,#python,nwreadinglist v4.4.0,True,True,2025,12


2015,↕,2016,↕,2017,↕,2018,↕,2019,↕,2020,↕,2021,↕,2022,↕,2023,↕,2024,↕,2025
00h 00m,↑,18h 00m,↑,88h 30m,↓,80h 15m,↓,60h 00m,↓,29h 15m,↑,53h 00m,↓,00h 00m,↑,06h 00m,↑,45h 45m,↑,79h 00m
00h 00m,↑,45h 30m,↑,65h 30m,↑,90h 45m,↓,73h 00m,↓,38h 00m,↓,31h 30m,↓,03h 00m,↑,24h 00m,↑,77h 45m,↓,74h 00m
00h 00m,↑,20h 45m,↑,71h 45m,↑,89h 00m,↓,75h 30m,↓,35h 00m,↑,40h 30m,↓,06h 15m,↑,50h 15m,↑,77h 45m,↓,38h 15m
00h 00m,↑,37h 30m,↑,68h 00m,↑,88h 30m,↓,59h 45m,↓,40h 45m,↓,19h 00m,↑,27h 30m,↓,19h 00m,↑,29h 30m,↑,103h 00m
00h 00m,↑,53h 00m,↑,83h 00m,↑,91h 15m,↓,54h 45m,↓,14h 30m,↑,112h 45m,↓,49h 45m,↓,31h 00m,↑,43h 00m,↑,107h 45m
00h 00m,↑,57h 45m,↓,37h 45m,↑,62h 00m,↓,29h 15m,↓,12h 00m,↑,54h 00m,↑,73h 30m,↓,24h 45m,↑,48h 00m,↑,104h 15m
00h 00m,↑,46h 45m,↑,65h 30m,↑,69h 30m,↓,24h 15m,↑,34h 00m,↓,23h 30m,↑,51h 00m,↓,16h 30m,↑,67h 00m,↑,132h 45m
00h 00m,↑,25h 45m,↑,45h 45m,↑,72h 00m,↓,06h 00m,↑,32h 00m,↑,110h 00m,↓,36h 30m,↑,41h 30m,↓,32h 45m,↑,82h 30m
00h 00m,↑,89h 30m,↓,43h 45m,↑,64h 00m,↓,39h 00m,↑,44h 00m,↓,43h 30m,↑,69h 00m,↓,50h 15m,↓,48h 00m,↑,58h 00m
08h 00m,↑,82h 15m,↓,64h 30m,↓,46h 45m,↓,45h 30m,↑,48h 00m,↓,35h 30m,↑,38h 30m,↓,20h 00m,↑,101h 30m,↓,67h 00m


2015,↕,2016,↕,2017,↕,2018,↕,2019,↕,2020,↕,2021,↕,2022,↕,2023,↕,2024,↕,2025
18h 00m,↑,615h 15m,↑,762h 45m,↑,829h 45m,↓,515h 15m,↓,470h 30m,↑,537h 30m,↓,467h 30m,↓,320h 15m,↑,862h 30m,↑,1086h 15m


11 Years
6485h 30m


SoftwareProjectName,Effort,Hashtags
nwknowledgebase,337h 15m,"#adoc, #python"
nwtraderaanalytics,263h 15m,#python
nwtimetracking,226h 15m,#python
NW.UnivariateForecasting,208h 00m,#csharp
NW.NGramTextClassification,207h 30m,#csharp
nwapolloanalytics,194h 00m,#duckdb
nwreadinglist,186h 00m,#python
nwrefurbishedanalytics,178h 00m,#python
nwpackageversions,56h 00m,#python
nwshared,47h 15m,#python


SoftwareProjectName,SoftwareProjectVersion,Effort
nwbuild,1.0.0,25h 45m
nwknowledgebase,1.0.0,337h 15m
nwrefurbishedanalytics,1.0.0,178h 00m


Hashtag,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024,2025
#adoc,,,,,,,,,,,327h 45m
#bash,,,,,,,,,,,20h 30m
#csharp,,,,,61h 45m,206h 30m,441h 45m,298h 45m,67h 30m,35h 15m,
#duckdb,,,,,,,,,,,194h 00m
#maintenance,,,,,53h 45m,53h 30m,29h 30m,36h 30m,52h 30m,117h 00m,158h 45m
#overtime,,,,,01h 00m,19h 00m,,,14h 30m,02h 00m,
#powershell,,,,,,116h 45m,11h 30m,21h 15m,04h 30m,,
#python,,,,,,,,72h 00m,126h 30m,577h 45m,292h 00m
#studying,,,,,75h 45m,74h 45m,54h 45m,39h 00m,54h 45m,130h 30m,93h 15m
#untagged,18h 00m,615h 15m,762h 45m,829h 45m,323h 00m,,,,,,


Hashtag,Effort,Effort%
#adoc,327h 45m,5.05
#bash,20h 30m,0.32
#csharp,1111h 30m,17.14
#duckdb,194h 00m,2.99
#maintenance,501h 30m,7.73
#overtime,36h 30m,0.56
#powershell,154h 00m,2.37
#python,1068h 15m,16.47
#studying,522h 45m,8.06
#untagged,2548h 45m,39.30


Year,Month,SoftwareProjectName,SoftwareProjectVersion,Effort
2025,1,nwknowledgebase,1.0.0,01h 30m
2025,4,nwknowledgebase,1.0.0,24h 15m
2025,5,nwbuild,1.0.0,25h 45m
2025,6,nwrefurbishedanalytics,1.0.0,64h 00m
2025,7,nwrefurbishedanalytics,1.0.0,113h 15m
2025,8,nwknowledgebase,1.0.0,15h 30m
2025,8,nwrefurbishedanalytics,1.0.0,00h 45m
2025,9,nwknowledgebase,1.0.0,58h 00m
2025,10,nwknowledgebase,1.0.0,59h 00m
2025,11,nwknowledgebase,1.0.0,71h 30m


Occurrences,Occurrence%,TimeRanges
71,22.33,['08:00-08:45']
37,11.64,['08:00-08:30']
26,8.18,['17:00-17:30']
22,6.92,"['17:15-17:45', '18:00-20:00']"
21,6.60,['17:30-18:00']
19,5.97,['19:00-20:00']
16,5.03,"['08:15-08:45', '17:00-17:45']"
15,4.72,['21:00-22:00']
13,4.09,['17:00-20:00']
12,3.77,['16:45-17:30']


#### Main : Debug

In [4]:
tt_processor.process_ttd_effort_status()

StartTime,EndTime,Effort,IsCorrect,Expected,Message
